In [1]:
%pwd

'e:\\Github Projects\\Retinal_Blood_Vessels_Segmentation\\notebooks'

In [2]:
import sys
sys.path.append("E:\\Github Projects\\Retinal_Blood_Vessels_Segmentation")

In [3]:
import os

import cv2

from patchify import patchify

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import (
    EarlyStopping,
    ModelCheckpoint,
    ReduceLROnPlateau
)
from tensorflow.keras.layers import (
    Input,
    BatchNormalization,
    Concatenate,
    Conv2D,
    Conv2DTranspose,
    Dropout,
    LeakyReLU,
    MaxPool2D
)
from tensorflow.keras.metrics import Recall, Precision
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

from scripts.prepare_data import convert_images_to_rgb

In [4]:
import tensorflow as tf
import numpy as np

def iou(y_true, y_pred):
    def f(y_true, y_pred):
        intersection = (y_true * y_pred).sum()
        union = y_true.sum() + y_pred.sum() - intersection
        x = (intersection + 1e-15) / (union + 1e-15)
        x = x.astype(np.float32)
        return x
    return tf.numpy_function(f, [y_true, y_pred], tf.float32)


def dice_coef(y_true, y_pred):
    smooth = 1e-15
    y_true = tf.keras.layers.Flatten()(y_true)
    y_pred = tf.keras.layers.Flatten()(y_pred)
    intersection = tf.reduce_sum(y_true * y_pred)
    return (2. * intersection + smooth) / (tf.reduce_sum(y_true) + tf.reduce_sum(y_pred) + smooth)

def dice_loss(y_true, y_pred):
    return 1.0 - dice_coef(y_true, y_pred)

In [11]:
def build_unet(IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS):
    inputs = Input((IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS))
    kernel_initializer =  'he_uniform'
    s = inputs

    c1 = Conv2D(16, (3, 3), activation='relu', kernel_initializer=kernel_initializer, padding='same')(s)
    c1 = Dropout(0.1)(c1)
    c1 = Conv2D(16, (3, 3), activation='relu', kernel_initializer=kernel_initializer, padding='same')(c1)
    p1 = MaxPool2D((2, 2))(c1)
    
    c2 = Conv2D(32, (3, 3), activation='relu', kernel_initializer=kernel_initializer, padding='same')(p1)
    c2 = Dropout(0.1)(c2)
    c2 = Conv2D(32, (3, 3), activation='relu', kernel_initializer=kernel_initializer, padding='same')(c2)
    p2 = MaxPool2D((2, 2))(c2)
     
    c3 = Conv2D(64, (3, 3), activation='relu', kernel_initializer=kernel_initializer, padding='same')(p2)
    c3 = Dropout(0.2)(c3)
    c3 = Conv2D(64, (3, 3), activation='relu', kernel_initializer=kernel_initializer, padding='same')(c3)
    p3 = MaxPool2D((2, 2))(c3)
     
    c4 = Conv2D(128, (3, 3), activation='relu', kernel_initializer=kernel_initializer, padding='same')(p3)
    c4 = Dropout(0.2)(c4)
    c4 = Conv2D(128, (3, 3), activation='relu', kernel_initializer=kernel_initializer, padding='same')(c4)
    p4 = MaxPool2D(pool_size=(2, 2))(c4)
     
    c5 = Conv2D(256, (3, 3), activation='relu', kernel_initializer=kernel_initializer, padding='same')(p4)
    c5 = Dropout(0.3)(c5)
    c5 = Conv2D(256, (3, 3), activation='relu', kernel_initializer=kernel_initializer, padding='same')(c5)
    
    
    u6 = Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(c5)
    u6 = Concatenate([u6, c4])
    c6 = Conv2D(128, (3, 3), activation='relu', kernel_initializer=kernel_initializer, padding='same')(u6)
    c6 = Dropout(0.2)(c6)
    c6 = Conv2D(128, (3, 3), activation='relu', kernel_initializer=kernel_initializer, padding='same')(c6)
     
    u7 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(c6)
    u7 = Concatenate([u7, c3])
    c7 = Conv2D(64, (3, 3), activation='relu', kernel_initializer=kernel_initializer, padding='same')(u7)
    c7 = Dropout(0.2)(c7)
    c7 = Conv2D(64, (3, 3), activation='relu', kernel_initializer=kernel_initializer, padding='same')(c7)
     
    u8 = Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(c7)
    u8 = Concatenate([u8, c2])
    c8 = Conv2D(32, (3, 3), activation='relu', kernel_initializer=kernel_initializer, padding='same')(u8)
    c8 = Dropout(0.1)(c8)
    c8 = Conv2D(32, (3, 3), activation='relu', kernel_initializer=kernel_initializer, padding='same')(c8)
     
    u9 = Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same')(c8)
    u9 = Concatenate([u9, c1], axis=3)
    c9 = Conv2D(16, (3, 3), activation='relu', kernel_initializer=kernel_initializer, padding='same')(u9)
    c9 = Dropout(0.1)(c9)
    c9 = Conv2D(16, (3, 3), activation='relu', kernel_initializer=kernel_initializer, padding='same')(c9)
     
    outputs = Conv2D(1, (1, 1), activation='sigmoid')(c9)  # maybe use softmax
     
    model = Model(inputs=[inputs], outputs=[outputs])
    model.compile(optimizer=Adam(lr = 1e-3), loss='binary_crossentropy', metrics=['accuracy'])  # maybe categorical_crossentropy
    # model.compile(optimizer=Adam(lr = 1e-3), loss=dice_loss, metrics=['accuracy', dice_coef])
    # model.summary()
    
    return model

In [ ]:
def make_patches(dir: str, shape: tuple) -> tuple:
    img_patch_shape = (*shape, 3)
    mask_patch_shape = shape
    """
    ewentualnie robic patche po 128, ale step=64. czesc by sie wtedy nakladala
    """

    X = [cv2.imread(f"{dir}/img/{img_name}") for img_name in os.listdir(dir)]
    X = convert_images_to_rgb(X)

    y = [cv2.imread(f"{dir}/mask/{img_name}", cv2.IMREAD_GRAYSCALE) for img_name in os.listdir(dir)]

    X_patchified = [patchify(img, img_patch_shape, step=img_patch_shape[0]) for img in X]
    y_patchified = [patchify(mask, mask_patch_shape, step=mask_patch_shape[0]) for mask in y]

    img_patches, mask_patches = [], []

    for patchified_image in X_patchified:
        for i in range(patchified_image.shape[0]):
            for j in range(patchified_image.shape[1]):
                img_patches.append(patchified_image[i, j, :, :][0])

    for patchified_mask in y_patchified:
        for i in range(patchified_image.shape[0]):
            for j in range(patchified_image.shape[1]):
                mask_patches.append(patchified_mask[i, j, :, :])
    

    return np.array(img_patches), np.array(mask_patches)

In [12]:
train_dir = "../cropped_images/train"
test_dir = "../cropped_images/test"

img_patch_shape = (64, 64, 3)
mask_patch_shape = (64, 64)

X_train, y_train = make_patches(train_dir, shape=(64, 64))
X_test, y_test = make_patches(test_dir, shape=(64, 64))

train_images_number = X_train.shape[0]
test_images_number = X_test.shape[0]
batch_size = 16

seed = 42

train_img_datagen_params = dict(rotation_range=90,
                            zoom_range=0.2,
                            horizontal_flip=True,
                            width_shift_range=0.3,
                            height_shift_range=0.3,
                            fill_mode='reflect',
                            rescale=1./255
)

train_mask_datagen_params = dict(rotation_range=90,
                            zoom_range=0.2,
                            horizontal_flip=True,
                            width_shift_range=0.3,
                            height_shift_range=0.3,
                            fill_mode='reflect',
                            preprocessing_function=lambda x: np.where(x>0, 1, 0).astype(x.dtype)
)

train_datagen_images = ImageDataGenerator(**train_img_datagen_params)
train_datagen_masks = ImageDataGenerator(**train_mask_datagen_params)

train_images = train_datagen_images.flow(X_train, batch_size=batch_size, seed=seed)
train_masks = train_datagen_masks.flow(y_train, batch_size=batch_size, seed=seed)

# train_images = train_datagen_images.flow_from_directory(directory=f"{train_dir}",
#                                                         target_size=(512, 512),
#                                                         batch_size=2,
#                                                         class_mode=None,
#                                                         classes=["img"],
#                                                         seed=seed)
# train_masks = train_datagen_masks.flow_from_directory(directory=f"{train_dir}",
#                                                       target_size=(512, 512),
#                                                       batch_size=2,
#                                                       class_mode=None,
#                                                       color_mode="grayscale",
#                                                       classes=["mask"],
#                                                       seed=seed)


test_datagen_params = dict(rescale=1./255)

test_datagen_images = ImageDataGenerator(**test_datagen_params)
test_datagen_masks = ImageDataGenerator(**test_datagen_params)

test_images = test_datagen_images.flow(X_test, batch_size=batch_size, seed=seed)
test_masks = test_datagen_masks.flow(y_test, batch_size=batch_size, seed=seed)

# test_images = test_datagen_images.flow_from_directory(directory=f"{test_dir}",
#                                                       target_size=(512, 512),
#                                                       class_mode=None,
#                                                       batch_size=2,
#                                                       classes=["img"],
#                                                       seed=seed)
# test_masks = test_datagen_masks.flow_from_directory(directory=f"{test_dir}",
#                                                     target_size=(512, 512),
#                                                     class_mode=None,
#                                                     batch_size=2,
#                                                     color_mode="grayscale",
#                                                     classes=["mask"],
#                                                     seed=seed)

train_set = zip(train_images, train_masks)
test_set = zip(test_images, test_masks)

Found 21 images belonging to 1 classes.
Found 21 images belonging to 1 classes.
Found 7 images belonging to 1 classes.
Found 7 images belonging to 1 classes.


In [13]:
model = build_unet(64, 64, 3)
model.summary()

Model: "UNET"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 512, 512, 3) 0                                            
__________________________________________________________________________________________________
conv2d_38 (Conv2D)              (None, 512, 512, 64) 1792        input_3[0][0]                    
__________________________________________________________________________________________________
batch_normalization_36 (BatchNo (None, 512, 512, 64) 256         conv2d_38[0][0]                  
__________________________________________________________________________________________________
leaky_re_lu_36 (LeakyReLU)      (None, 512, 512, 64) 0           batch_normalization_36[0][0]     
_______________________________________________________________________________________________

In [14]:
callbacks = [
    ModelCheckpoint(filepath="model-{epoch:02d}-{val_loss:.2f}.hdf5",
                    monitor="val_loss",
                    save_best_only=True),
    EarlyStopping(monitor="val_loss",
                  patience=10,
                  restore_best_weights=True),
    ReduceLROnPlateau(monitor="val_loss",
                      factor=0.1,
                      patience=5,
                      verbose=1,
                      min_lr=1e-6)
]

# model.compile(loss=dice_loss, optimizer=Adam(learning_rate=1e-4), metrics=[dice_coef, iou, Recall(), Precision()])

In [15]:
# model.fit(train_set,
#           validation_data=test_set,
#           epochs=10,
#           batch_size=2,
#           callbacks=callbacks)
model.fit(train_set,
          validation_data=test_set,
          epochs=100,
          steps_per_epoch=train_images_number//batch_size,
          validation_steps=test_images_number//batch_size,
          callbacks=callbacks)

Epoch 1/10


ResourceExhaustedError: 2 root error(s) found.
  (0) Resource exhausted:  OOM when allocating tensor with shape[2,256,256,256] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[node UNET/concatenate_9/concat (defined at <ipython-input-15-c9765644046b>:6) ]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

	 [[assert_less_equal_1/Assert/AssertGuard/pivot_f/_41/_89]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

  (1) Resource exhausted:  OOM when allocating tensor with shape[2,256,256,256] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[node UNET/concatenate_9/concat (defined at <ipython-input-15-c9765644046b>:6) ]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_

In [ ]:
for i in range(30):
    print(i)
    print(next(train_images))